### The Team Name: TheProfessor
Luong Bang Tran - 1956417

Gaurav Mohan Ramse - 1965564

Tansel Simsek - 1942297


In [ ]:
pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L12-v2')

In [5]:
import pandas as pd
data = pd.read_csv('/content/EfficentNetB7_20ep_glove.csv')
data

,true_caption,pred_caption
0,a white canoe in a body of water in front of a...,a duck perched on a pier
1,a man kiteboarding over waves in the ocean,two people standing in the sun
2,a team playing baseball on a baseball diamond,some baseball play gets read to pitch the ball...
3,a group of players <unk> as they reach for a f...,everyone playing with a horse
4,a dog following two people on horses on a beach,a man riding a horse are on a horse pulling a ...
...,...,...
995,a body of water filled with lots of boats,the two boats are docked at the river
996,a table full of various types of cellphones <u...,some items that are stopped on the ground with...
997,several people teach a child how to surf,a group of people that are smiling <unk>
998,some people are on a large wooden float shaped...,a group of people <unk> with everything


In [6]:
data['true_caption_emdedding'] = data['true_caption'].apply(lambda x : model.encode(x))

In [7]:
data['pred_caption_emdedding'] = data['pred_caption'].apply(lambda x : model.encode(x))

In [8]:
data

,true_caption,pred_caption,true_caption_emdedding,pred_caption_emdedding
0,a white canoe in a body of water in front of a...,a duck perched on a pier,"[0.3325414, 0.19329476, 0.05953163, -0.3599656...","[0.3931591, -0.17856348, 0.34539768, -0.318751..."
1,a man kiteboarding over waves in the ocean,two people standing in the sun,"[-0.44295883, -0.014238959, 0.24918152, -0.142...","[-0.08283384, 0.47484773, 0.03006237, 0.428543..."
2,a team playing baseball on a baseball diamond,some baseball play gets read to pitch the ball...,"[0.22418919, -0.031759262, -0.63223886, -0.189...","[0.13959636, 0.17563227, -0.4827478, -0.327531..."
3,a group of players <unk> as they reach for a f...,everyone playing with a horse,"[-0.012096699, 0.09747802, -0.06521227, -0.322...","[0.29466492, 0.36850715, -0.16702132, -0.08677..."
4,a dog following two people on horses on a beach,a man riding a horse are on a horse pulling a ...,"[-0.030751439, -0.23862481, 0.13338763, 0.0448...","[0.42211467, 0.24397019, -0.21555415, 0.112046..."
...,...,...,...,...
995,a body of water filled with lots of boats,the two boats are docked at the river,"[0.6959337, -0.25576136, 0.432085, 0.21343961,...","[0.3679857, -0.038122468, 0.25713116, 0.016179..."
996,a table full of various types of cellphones <u...,some items that are stopped on the ground with...,"[-0.22326986, 0.307643, -0.34172833, -0.005255...","[-0.15865888, 0.36322302, 0.2069868, -0.211507..."
997,several people teach a child how to surf,a group of people that are smiling <unk>,"[0.2647838, -0.13652126, 0.60782015, -0.572181...","[0.238511, 0.23785415, -0.09250707, -0.212799,..."
998,some people are on a large wooden float shaped...,a group of people <unk> with everything,"[0.36809057, 0.14977951, -0.13179627, -0.26859...","[-0.0016979658, 0.23372246, -0.33724484, -0.08..."


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval
def get_similar_result_cosine(idx, real_captions, pred_captions):
    s_score_list = []

    for idx_2 in range(len(pred_captions)):
      
      s_score = cosine_similarity([real_captions[idx]], [pred_captions[idx_2]])[0][0]
    #   print(s_score)
      s_score_list.append((idx_2, s_score))

    s_score_list.sort(key=lambda x: x[1], reverse=True)

    return s_score_list

In [10]:
def create_submission_file(top_k, real_captions, pred_captions):
    with open('./submission_jaccard.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerow(["caption", "image_list"])

        for idx in range(0,1000):

            s_score_res = get_similar_result_cosine(idx, real_captions, pred_captions)
            # print(s_score_res)
            writer.writerow([real_captions[idx],' '.join(list(map(lambda x: str(x[0]), s_score_res[:1000])))])

In [11]:
import csv
import tqdm
import numpy as np
create_submission_file(1000, data['true_caption_emdedding'].tolist(), data['pred_caption_emdedding'].tolist())